In [4]:
from train_contrastive import load_data

In [5]:
path = "../data/tabula_muris/preprocessed/tm_adata_test.pkl"
path_r = "../data/tabula_muris/preprocessed_reduced/tm_adata_test.pkl"

x, domains, cells, adata = load_data(path)
x_r, domains_r, cells_r, adata_r = load_data(path_r)

In [7]:
from contrastive_model import ContrastiveModel

CONTRASTIVE_TRANSFORMER_KEY="contrastive_transformer"
CONTRASTIVE_MLP_KEY="contrastive_mlp"

c_transformer_path = "runs/attention_05-10-23:02/final.ckpt"
c_mlp_path = "runs/dense_05-11-12:36/final.ckpt"


c_transformer = ContrastiveModel.load_from_checkpoint(c_transformer_path)
c_mlp = ContrastiveModel.load_from_checkpoint(c_mlp_path)

In [13]:
import torch
from tqdm import tqdm

batch_size = 32  # Define a suitable batch size
ct_embeddings_list = []
cm_embeddings_list = []

c_transformer.eval()
c_mlp.eval()

x = torch.tensor(x, dtype=torch.float32)
x_r = torch.tensor(x_r, dtype=torch.float32)


with torch.no_grad():
    for i in tqdm(range(0, x.size(0), batch_size), desc="Processing Batches"):
        x_batch = x[i:i + batch_size]
        x_r_batch = x_r[i:i + batch_size]
        
        ct_embeddings_batch = c_transformer.encoder(x_r_batch)
        cm_embeddings_batch = c_mlp.encoder(x_batch)
        
        ct_embeddings_list.append(ct_embeddings_batch)
        cm_embeddings_list.append(cm_embeddings_batch)

ct_embeddings = torch.cat(ct_embeddings_list, dim=0)
cm_embeddings = torch.cat(cm_embeddings_list, dim=0)

Processing Batches: 100%|███████████████████████████████████████████████████████████| 1931/1931 [16:47<00:00,  1.92it/s]


In [14]:
adata.obsm[CONTRASTIVE_TRANSFORMER_KEY] = ct_embeddings.cpu().numpy()
adata.obsm[CONTRASTIVE_MLP_KEY] = cm_embeddings.cpu().numpy()

/tmp/ipykernel_1927599/491995230.py:1: ImplicitModificationWarning: Setting element `.obsm['contrastive_transformer']` of view, initializing view as actual.
  adata.obsm[CONTRASTIVE_TRANSFORMER_KEY] = ct_embeddings.cpu().numpy()


In [15]:
import scanpy as sc

In [16]:
sc.pp.neighbors(adata, use_rep=CONTRASTIVE_TRANSFORMER_KEY)  # Compute neighbors using latent space
sc.tl.umap(adata)
fig1 = sc.pl.umap(adata, color='tech', legend_loc=None, title='', frameon=False, size=50, return_fig=True, show=True)
fig1.savefig('umap_transformer_final.png')

/oscar/rt/9.2/software/0.20-generic/0.20.1/opt/spack/linux-rhel9-x86_64_v3/gcc-11.3.1/anaconda-2023.09-0-7nso27ys7navjquejqdxqylhg7kuyvxo/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [17]:
sc.pp.neighbors(adata, use_rep=CONTRASTIVE_MLP_KEY)  # Compute neighbors using latent space
sc.tl.umap(adata)
fig = sc.pl.umap(adata, color='tech', legend_loc=None, title='', frameon=False, size=50, return_fig=True, show=True)
fig.savefig('umap_mlp_final.png')

In [18]:
from scib_metrics.benchmark import Benchmarker
bm = Benchmarker(
    adata,
    batch_key='tech',
    label_key='cell_ontology_class',
    embedding_obsm_keys=[CONTRASTIVE_TRANSFORMER_KEY, CONTRASTIVE_MLP_KEY],
    n_jobs=-1
)
bm.benchmark()

/users/ndepiero/.local/lib/python3.11/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
Computing neighbors:   0%|                                                                        | 0/2 [00:00<?, ?it/s]

Embeddings:   0%|                                                                                 | 0/2 [00:00<?, ?it/s]

INFO     35 clusters consist of a single batch or are too small. Skip.                                             


Embeddings:  50%|█████████████████████████████████▌                                 | 1/2 [1:02:40<1:02:40, 3760.05s/it]

INFO     35 clusters consist of a single batch or are too small. Skip.                                             


/users/ndepiero/.local/lib/python3.11/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(
/users/ndepiero/.local/lib/python3.11/site-packages/scib_metrics/metrics/_pcr_comparison.py:49: UserWarning: PCR comparison score is negative, meaning variance contribution increased after integration. Setting to 0.
  warnings.warn(
Embeddings: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [1:57:16<00:00, 3518.01s/it]
